In [35]:
file = sc.textFile('file:///C:/Users/Ashwini/Desktop/onTimeDepartPerf')


In [21]:
file.count()

2965

In [36]:
file = file.map(lambda line:line.split()).filter(lambda line:len(line)==3)
file.take(10)

[['ABE', '9E', '6.771714922048997'],
 ['ABE', 'AA', '4.630407124681934'],
 ['ABE', 'DH', '5.356243252968694'],
 ['ABE', 'DL', '23.120029967356984'],
 ['ABE', 'EA', '6.578622482131254'],
 ['ABE', 'EV', '15.46139963906156'],
 ['ABE', 'NW', '1.8226794590220783'],
 ['ABE', 'OH', '6.53022997032641'],
 ['ABE', 'OO', '10.096487686717804'],
 ['ABE', 'PI', '5.016528925619835']]

In [39]:
file1 = file.map(lambda tuple:(tuple[0],tuple[1],float(tuple[2])))


In [40]:
file1.count()

2882

In [42]:
from pyspark.sql import Row
rdd = file1.map(lambda row: Row(airport=row[0], carrier=row[1], dep_delay=row[2]))


In [43]:
df = spark.createDataFrame(rdd)

In [44]:
df.show()

+-------+-------+-------------------+
|airport|carrier|          dep_delay|
+-------+-------+-------------------+
|    ABE|     9E|  6.771714922048997|
|    ABE|     AA|  4.630407124681934|
|    ABE|     DH|  5.356243252968694|
|    ABE|     DL| 23.120029967356984|
|    ABE|     EA|  6.578622482131254|
|    ABE|     EV|  15.46139963906156|
|    ABE|     NW| 1.8226794590220783|
|    ABE|     OH|   6.53022997032641|
|    ABE|     OO| 10.096487686717804|
|    ABE|     PI|  5.016528925619835|
|    ABE|     TW|  2.002008032128514|
|    ABE|     UA|  8.764681724845996|
|    ABE|     US| 2.6949712267852473|
|    ABE|     XE|-3.8152038454691115|
|    ABE|     YV| 21.996439697374278|
|    ABI|     EV|               1.75|
|    ABI|     MQ|  3.869399156310569|
|    ABI|     OO|  6.579889807162535|
|    ABI|     US|                0.0|
|    ABI|     XE| -4.604166666666667|
+-------+-------+-------------------+
only showing top 20 rows



In [80]:
from pyspark.sql.functions import ceil, col,round


In [49]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])  # provide contact points and port
session = cluster.connect('aviation')

In [81]:
df1 = df.select("airport","carrier",round(col('dep_delay'),4))

In [64]:
df1.show()

+-------+-------+-------------------+
|airport|carrier|round(dep_delay, 4)|
+-------+-------+-------------------+
|    ABE|     9E|             6.7717|
|    ABE|     AA|             4.6304|
|    ABE|     DH|             5.3562|
|    ABE|     DL|              23.12|
|    ABE|     EA|             6.5786|
|    ABE|     EV|            15.4614|
|    ABE|     NW|             1.8227|
|    ABE|     OH|             6.5302|
|    ABE|     OO|            10.0965|
|    ABE|     PI|             5.0165|
|    ABE|     TW|              2.002|
|    ABE|     UA|             8.7647|
|    ABE|     US|              2.695|
|    ABE|     XE|            -3.8152|
|    ABE|     YV|            21.9964|
|    ABI|     EV|               1.75|
|    ABI|     MQ|             3.8694|
|    ABI|     OO|             6.5799|
|    ABI|     US|                0.0|
|    ABI|     XE|            -4.6042|
+-------+-------+-------------------+
only showing top 20 rows



In [65]:
df1 =df1.withColumnRenamed("round(dep_delay, 4)","dep_delay")

In [66]:
df1.show()

+-------+-------+---------+
|airport|carrier|dep_delay|
+-------+-------+---------+
|    ABE|     9E|   6.7717|
|    ABE|     AA|   4.6304|
|    ABE|     DH|   5.3562|
|    ABE|     DL|    23.12|
|    ABE|     EA|   6.5786|
|    ABE|     EV|  15.4614|
|    ABE|     NW|   1.8227|
|    ABE|     OH|   6.5302|
|    ABE|     OO|  10.0965|
|    ABE|     PI|   5.0165|
|    ABE|     TW|    2.002|
|    ABE|     UA|   8.7647|
|    ABE|     US|    2.695|
|    ABE|     XE|  -3.8152|
|    ABE|     YV|  21.9964|
|    ABI|     EV|     1.75|
|    ABI|     MQ|   3.8694|
|    ABI|     OO|   6.5799|
|    ABI|     US|      0.0|
|    ABI|     XE|  -4.6042|
+-------+-------+---------+
only showing top 20 rows



In [68]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)

In [77]:
df1.write\
.format("org.apache.spark.sql.cassandra")\
.mode('append')\
.options(table = "carrier_depart_delay", keyspace = "aviation")  \
.save()

In [78]:
data= ds = sqlContext.read \
  .format('org.apache.spark.sql.cassandra') \
  .options(table='carrier_depart_delay', keyspace='aviation') \
  .load()

In [79]:
data.show()

+-------+--------------------+-------+
|airport|           dep_delay|carrier|
+-------+--------------------+-------+
|    LYH|6.858600000000000000|     EV|
|    LYH|5.754900000000000000|     PI|
|    LYH|2.895000000000000000|     US|
|    CRW|16.60420000000000...|     OO|
|    CRW|12.62950000000000...|     EV|
|    CRW|8.560000000000000000|     YV|
|    CRW|5.742200000000000000|     PI|
|    CRW|5.238400000000000000|     US|
|    CRW|4.631900000000000000|     DH|
|    CRW|4.419300000000000000|     XE|
|    CRW|2.731700000000000000|     OH|
|    CRW|-0.13330000000000...|     UA|
|    CRW|-0.14150000000000...|     9E|
|    MFR|17.07430000000000...|     CO|
|    MFR|12.04590000000000...|     UA|
|    MFR|8.538000000000000000|     OO|
|    MFR|8.379900000000000000|     PS|
|    MFR|6.500800000000000000|     US|
|    MFR|4.050700000000000000|     YV|
|    ISP|18.02280000000000...|     EV|
+-------+--------------------+-------+
only showing top 20 rows

